In [1]:
from langchain_groq import ChatGroq

In [2]:
llm = ChatGroq(
    temperature=0, 
    groq_api_key='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', 
    model_name="llama-3.1-70b-versatile"
)
response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped onto the lunar surface on July 20, 1969, as part of the Apollo 11 mission.


In [19]:
#to scrape the data of job postings from a website

from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.amazon.jobs/en/jobs/2802318/software-dev-engineer-iii-aws-payments")
page_data = loader.load().pop().page_content
#print(page_data)

In [20]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
print(res.content)

```json
{
  "role": "Software Dev Engineer III, AWS Payments",
  "experience": "5+ years",
  "skills": [
    "programming with at least one software programming language",
    "leading design or architecture (design patterns, reliability and scaling) of new and existing systems",
    "full software development life cycle, including coding standards, code reviews, source control management, build processes, testing, and operations",
    "cloud computing",
    "machine learning",
    "designing highly scalable, fault tolerant systems"
  ],
  "description": "The AWS Commerce Platform Payments team is looking for talented and passionate software development engineers with strong technical and business systems backgrounds to build new invoicing, payments and financial reconciliation services and products."
}
```


In [21]:

from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Software Dev Engineer III, AWS Payments',
 'experience': '5+ years',
 'skills': ['programming with at least one software programming language',
  'leading design or architecture (design patterns, reliability and scaling) of new and existing systems',
  'full software development life cycle, including coding standards, code reviews, source control management, build processes, testing, and operations',
  'cloud computing',
  'machine learning',
  'designing highly scalable, fault tolerant systems'],
 'description': 'The AWS Commerce Platform Payments team is looking for talented and passionate software development engineers with strong technical and business systems backgrounds to build new invoicing, payments and financial reconciliation services and products.'}

In [22]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [23]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [25]:
job = json_res
job['skills']


['programming with at least one software programming language',
 'leading design or architecture (design patterns, reliability and scaling) of new and existing systems',
 'full software development life cycle, including coding standards, code reviews, source control management, build processes, testing, and operations',
 'cloud computing',
 'machine learning',
 'designing highly scalable, fault tolerant systems']

In [26]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/android-portfolio'},
  {'links': 'https://example.com/java-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/java-portfolio'}],
 [{'links': 'https://example.com/xamarin-portfolio'},
  {'links': 'https://example.com/devops-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/android-portfolio'},
  {'links': 'https://example.com/flutter-portfolio'}]]

In [29]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Pranav, a business development executive at ABC Ltd. ABC is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of ABC 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase ABC's portfolio: {link_list}
        Remember you are Pranav, BDE at ABC. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Solutions for AWS Payments Platform Development

Dear Hiring Manager,

I came across the job posting for a Software Dev Engineer III, AWS Payments, and I'm excited to introduce ABC Ltd. as a potential partner to help you build new invoicing, payments, and financial reconciliation services and products.

With over 5+ years of experience in software development, our team at ABC Ltd. has a strong technical and business systems background, making us an ideal fit for this project. Our expertise in programming languages, leading design or architecture, and full software development life cycle will enable us to deliver high-quality solutions that meet your requirements.

Our team has extensive experience in cloud computing, machine learning, and designing highly scalable, fault-tolerant systems. We've worked with various clients across industries, empowering them with tailored solutions that foster scalability, process optimization, cost reduction, and heightened overall effic